In [1]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import pysqkit as sq
from itertools import product, combinations
from matplotlib.lines import Line2D
from IPython.display import display, Latex, Image

In [2]:
fluxonium_A = sq.qubits.Fluxonium(
    label='A',
    joseph_energy=5.899, 
    charge_energy=0.973, 
    induct_energy=0.457,
    flux = 0.5
)

fluxonium_B = sq.qubits.Fluxonium(
    label='B',
    joseph_energy=5.768, 
    charge_energy=1.027, 
    induct_energy=0.684,
    flux = 0.5
)

fluxonium_A.diagonalize_basis(5)
fluxonium_B.diagonalize_basis(5)

In [3]:
qubit_A_tran_freqs = np.diff(fluxonium_A.eig_energies(), 1)
for level in range(3):
    display(Latex(r'$\omega^{{f}}_{{{} \rightarrow {}}}$ = {:.3f} GHz'.format(level, level+1, qubit_A_tran_freqs[level])))

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

In [4]:
qubit_B_tran_freqs = np.diff(fluxonium_B.eig_energies(), 1)
for level in range(3):
    display(Latex(r'$\omega^{{f}}_{{{} \rightarrow {}}}$ = {:.3f} GHz'.format(level, level+1, qubit_B_tran_freqs[level])))

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

In [5]:
system = fluxonium_A.couple_to(fluxonium_B, coupling=sq.couplers.capacitive_coupling, strength=0.224)

In [11]:
comp_states = [''.join(qubit_states) for qubit_states in list(product('012', repeat=2))]
q_labels = [qubit.label for qubit in system]

level_inds = xr.DataArray([system.state_index(state) for state in comp_states], dims=['state'], coords=dict(state=comp_states))
level_inds = level_inds.sortby(level_inds)

eig_energies = system.eig_energies(levels=level_inds.data)

trans_freqs = xr.DataArray(
    eig_energies[:,np.newaxis] - eig_energies, 
    dims=['out_state', 'in_state'], 
    coords=dict(
        out_state=level_inds.state.data, 
        in_state=level_inds.state.data
    )
)

mat_elem_amps = xr.DataArray(
    [system.mat_elements(qubit.charge_op(), levels=level_inds.data) for qubit in system],
    dims=['qubit', 'out_state', 'in_state'], 
    coords=dict( 
        qubit=q_labels,
        out_state=level_inds.state.data, 
        in_state=level_inds.state.data
    )
)

In [23]:
trans_freq_11_12 = float(trans_freqs.sel(in_state='11', out_state='21'))
trans_freq_01_02 = float(trans_freqs.sel(in_state='10', out_state='20'))
freq_split = trans_freq_11_12 - trans_freq_01_02
display(Latex(r'$\Delta$ = {:.3f} GHz'.format(freq_split)))

drive_str_ratio = 0.9
drive_str_B = 0.5
drive_str_A = drive_str_ratio*drive_str_B

rabi_freq_11_21 = float(np.abs(drive_str_A*mat_elem_amps.sel(qubit='A', in_state='11', out_state='21') + drive_str_B*mat_elem_amps.sel(qubit='B', in_state='11', out_state='21')))
rabi_freq_10_20 = float(np.abs(drive_str_A*mat_elem_amps.sel(qubit='A', in_state='10', out_state='20') + drive_str_B*mat_elem_amps.sel(qubit='B', in_state='10', out_state='20')))

rabi_freq_ratio = rabi_freq_11_21/rabi_freq_10_20
display(Latex(r'$\Omega_{{11-21}}/\Omega_{{10-20}}$ = {:.3f}'.format(rabi_freq_ratio)))

_r2 = rabi_freq_ratio**2
drive_freq_ratio = (_r2 - np.sqrt((_r2-1)**2 + _r2))/(_r2-1)
display(Latex(r'$\frac{{\delta}}{{\Delta}}$ = {:.3f}'.format(drive_freq_ratio)))

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>